## 학습목표
 1. dataframe 제공 함수를 통한 데이터 처리 기법 숙지
 2. groupby, concat, merge, pivot, pivot_table, stack, unstack, transorm 등

### group by
  + 아래의 세 단계를 포함 (SQL의 group by 와 개념적으로는 동일, 사용법은 유사)
    - 데이터 분할
    - operation 적용
    - 데이터 병합

In [24]:
import pandas as pd
import numpy as np

In [25]:
# 1955년부터의 유로리그 우승팀 데이터
df = pd.read_csv('data/euro_winners.csv')

In [26]:
df.head(10)

,Season,Nation,Winners,Score,Runners-up,Runner-UpNation,Venue,Attendance
0,1955–56,Spain,Real Madrid,4–3,Stade de Reims,France,"Parc des Princes,Paris",38239
1,1956–57,Spain,Real Madrid,2–0,Fiorentina,Italy,"Santiago Bernabéu Stadium, Madrid",124000
2,1957–58,Spain,Real Madrid,3–2,Milan,Italy,"Heysel Stadium,Brussels",67000
3,1958–59,Spain,Real Madrid,2–0,Stade de Reims,France,"Neckarstadion,Stuttgart",72000
4,1959–60,Spain,Real Madrid,7–3,Eintracht Frankfurt,Germany,"Hampden Park,Glasgow",127621
5,1960–61,Portugal,Benfica,3–2,Barcelona,Spain,"Wankdorf Stadium,Bern",26732
6,1961–62,Portugal,Benfica,5–3,Real Madrid,Spain,"Olympisch Stadion,Amsterdam",61257
7,1962–63,Italy,Milan,2–1,Benfica,Portugal,"Wembley Stadium,London",45715
8,1963–64,Italy,Internazionale,3–1,Real Madrid,Spain,"Prater Stadium,Vienna",71333
9,1964–65,Italy,Internazionale,1–0,Benfica,Portugal,"San Siro, Milan",89000


In [27]:
# 국가별로 그룹핑 
nation_group = df.groupby('Nation')
print(nation_group)

In [28]:
type(nation_group)

pandas.core.groupby.DataFrameGroupBy

In [29]:
nation_group.groups # groups 속성으로 group 정보를 dict로 반환 ## 인덱스번호로 반환해줌 

{'England': Int64Index([12, 21, 22, 23, 24, 25, 26, 28, 43, 49, 52, 56], dtype='int64'),
 'France': Int64Index([37], dtype='int64'),
 'Germany': Int64Index([18, 19, 20, 27, 41, 45, 57], dtype='int64'),
 'Italy': Int64Index([7, 8, 9, 13, 29, 33, 34, 38, 40, 47, 51, 54], dtype='int64'),
 'Netherlands': Int64Index([14, 15, 16, 17, 32, 39], dtype='int64'),
 'Portugal': Int64Index([5, 6, 31, 48], dtype='int64'),
 'Romania': Int64Index([30], dtype='int64'),
 'Scotland': Int64Index([11], dtype='int64'),
 'Spain': Int64Index([0, 1, 2, 3, 4, 10, 36, 42, 44, 46, 50, 53, 55], dtype='int64'),
 'Yugoslavia': Int64Index([35], dtype='int64')}

In [30]:
len(nation_group.groups) # 그룹의 개수

10

* group 함수
 - count, size 등

In [31]:
nation_group.count() ## 데이타 프레임으로 반환 

,Season,Winners,Score,Runners-up,Runner-UpNation,Venue,Attendance
Nation,,,,,,,
England,12,12,12,12,12,12,12
France,1,1,1,1,1,1,1
Germany,7,7,7,7,7,7,7
Italy,12,12,12,12,12,12,12
Netherlands,6,6,6,6,6,6,6
Portugal,4,4,4,4,4,4,4
Romania,1,1,1,1,1,1,1
Scotland,1,1,1,1,1,1,1
Spain,13,13,13,13,13,13,13


In [32]:
nation_group.size() ## 그룹별로 시리즈 형식으로 반환 

Nation
England        12
France          1
Germany         7
Italy          12
Netherlands     6
Portugal        4
Romania         1
Scotland        1
Spain          13
Yugoslavia      1
dtype: int64

In [33]:
nation_group.size().sort_values(ascending = False)

Nation
Spain          13
Italy          12
England        12
Germany         7
Netherlands     6
Portugal        4
Yugoslavia      1
Scotland        1
Romania         1
France          1
dtype: int64

In [34]:
# 2개 이상의 컬럼으로 grouping 가능
club_group = df.groupby(['Nation', 'Winners'])
club_group.size().sort_values(ascending = False)

Nation       Winners          
Spain        Real Madrid          9
Italy        Milan                7
Germany      Bayern Munich        5
England      Liverpool            5
Spain        Barcelona            4
Netherlands  Ajax                 4
England      Manchester United    3
Italy        Internazionale       3
             Juventus             2
Portugal     Porto                2
             Benfica              2
England      Nottingham Forest    2
             Chelsea              1
France       Marseille            1
Yugoslavia   Red Star Belgrade    1
Germany      Borussia Dortmund    1
             Hamburg              1
Netherlands  Feyenoord            1
             PSV Eindhoven        1
Romania      Steaua Bucure?ti     1
Scotland     Celtic               1
England      Aston Villa          1
dtype: int64

* 각 리그별 경기수, 득점 데이터

In [35]:
df = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [36]:
df.head(20)

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0
5,01/01/2013,MatchesPlayed,42.0,40,40.0,18.0
6,02/01/2013,MatchesPlayed,30.0,40,40.0,36.0
7,03/01/2013,MatchesPlayed,35.0,38,39.0,36.0
8,04/01/2013,MatchesPlayed,42.0,42,41.0,36.0
9,05/01/2013,MatchesPlayed,33.0,40,40.0,27.0


* inplace parameter
 - 대부분의 dataframe 연산은 원본데이터를 유지하고 복사본을 반환
 - 원본 자체를 변경하고 싶으면 inplace paramter를 True로 설정

In [37]:
# Month column을 index로 생성
df = df.set_index('Month') ## inplace 가 false 이므로 원본이 아닌 카피본으로 셋팅

# 혹은 아래와 같이 가능
#df.set_index(['Month', 'Stat'], inplace=True)

In [38]:
df.head() ## month는 인덱스화 되어서 컬럼이아님 .. 내려와서 인덱스로 명시됨 

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [39]:
# 다시 index를 column 레벨로 변경
df = df.reset_index() ## 인덱스 번호 (원본보호, 복사본 반환후 원본 다시 reset 해줌 )

In [40]:
df.head(10)

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0
5,01/01/2013,MatchesPlayed,42.0,40,40.0,18.0
6,02/01/2013,MatchesPlayed,30.0,40,40.0,36.0
7,03/01/2013,MatchesPlayed,35.0,38,39.0,36.0
8,04/01/2013,MatchesPlayed,42.0,42,41.0,36.0
9,05/01/2013,MatchesPlayed,33.0,40,40.0,27.0


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
Month         22 non-null object
Stat          22 non-null object
EPL           20 non-null float64
La Liga       22 non-null int64
Serie A       20 non-null float64
Bundesliga    20 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.1+ KB


In [42]:
df.Month[0].split('/')

['08', '01', '2012']

* groupby에 함수 사용

In [20]:
df.set_index(['Month'], inplace=True)


KeyError: 'Month'

In [198]:
# group by year
# 함수가 전달되는 경우 함수의 파라미터로 index 값이 전달
year_group = df.groupby(lambda month : month.split('/')[2]) ## 2013년


In [199]:
year_group.groups

{'2012': Index(['08/01/2012', '09/01/2012', '10/01/2012', '11/01/2012', '12/01/2012',
        '08/01/2012', '09/01/2012', '10/01/2012', '11/01/2012', '12/01/2012'],
       dtype='object', name='Month'),
 '2013': Index(['01/01/2013', '02/01/2013', '03/01/2013', '04/01/2013', '05/01/2013',
        '06/02/2013', '01/01/2013', '02/01/2013', '03/01/2013', '04/01/2013',
        '05/01/2013', '06/01/2013'],
       dtype='object', name='Month')}

In [200]:
year_group.size()

2012    10
2013    12
dtype: int64

In [201]:
# index의 경우에는 column name이 아닌, 레벨로 grouping 가능
month_group = df.groupby(level = 0)

In [202]:
# 혹은 다음과 같이 index 자체를 명시 가능
month_group = df.groupby(level = 'Month')

In [45]:
for name, group in month_group:
    print(name, '\n')
    print(group)

0 

   level_0  index       Month           Stat   EPL  La Liga  Serie A  \
0        0      0  08/01/2012  MatchesPlayed  20.0       20     10.0   

   Bundesliga  
0        10.0  
1 

   level_0  index       Month           Stat   EPL  La Liga  Serie A  \
1        1      1  09/01/2012  MatchesPlayed  38.0       39     50.0   

   Bundesliga  
1        44.0  
2 

   level_0  index       Month           Stat   EPL  La Liga  Serie A  \
2        2      2  10/01/2012  MatchesPlayed  31.0       31     39.0   

   Bundesliga  
2        27.0  
3 

   level_0  index       Month           Stat   EPL  La Liga  Serie A  \
3        3      3  11/01/2012  MatchesPlayed  50.0       41     42.0   

   Bundesliga  
3        46.0  
4 

   level_0  index       Month           Stat   EPL  La Liga  Serie A  \
4        4      4  12/01/2012  MatchesPlayed  59.0       39     39.0   

   Bundesliga  
4        26.0  
5 

   level_0  index       Month           Stat   EPL  La Liga  Serie A  \
5        5      5  

### multi index grouping

* index 항목 제거

In [203]:
df.reset_index(inplace=True)

In [204]:
df.head()

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.00,20,10.00,10.00
1,09/01/2012,MatchesPlayed,38.00,39,50.00,44.00
2,10/01/2012,MatchesPlayed,31.00,31,39.00,27.00
3,11/01/2012,MatchesPlayed,50.00,41,42.00,46.00
4,12/01/2012,MatchesPlayed,59.00,39,39.00,26.00


In [255]:
df.shape

(22, 6)

* 2가지 컬럼으로 인덱싱

In [205]:
# 멀티 레벨 인덱스를 생성
df = df.set_index(['Month', 'Stat'])
df.head()

,,EPL,La Liga,Serie A,Bundesliga
Month,Stat,,,,
08/01/2012,MatchesPlayed,20.00,20,10.00,10.00
09/01/2012,MatchesPlayed,38.00,39,50.00,44.00
10/01/2012,MatchesPlayed,31.00,31,39.00,27.00
11/01/2012,MatchesPlayed,50.00,41,42.00,46.00
12/01/2012,MatchesPlayed,59.00,39,39.00,26.00


* 멀티 인덱스에서 row 선택

In [206]:
df.loc['08/01/2012']

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
MatchesPlayed,20.00,20,10.00,10.00
GoalsScored,57.00,60,21.00,23.00


In [50]:
df.loc['08/01/2012'].loc['MatchesPlayed']

level_0        0.0
index          0.0
EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: MatchesPlayed, dtype: float64

In [51]:
df.loc['08/01/2012', 'MatchesPlayed']

level_0        0.0
index          0.0
EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: (08/01/2012, MatchesPlayed), dtype: float64

* 멀티 인덱스의 레벨을 이용한 그룹핑

In [54]:
# multi level grouping
month_stat_group = df.groupby(level = [0, 1])

In [53]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)

('01/01/2013', 'GoalsScored')
                        level_0  index    EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                            
01/01/2013 GoalsScored       16     16  117.0      121    104.0        51.0
('01/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
01/01/2013 MatchesPlayed        5      5  42.0       40     40.0        18.0
('02/01/2013', 'GoalsScored')
                        level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                           
02/01/2013 GoalsScored       17     17  87.0      110    100.0       101.0
('02/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
02/01/2013 MatchesPlayed        6     

In [55]:
month_stat_group = df.groupby(level = ['Month', 'Stat'])

In [56]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)

('01/01/2013', 'GoalsScored')
                        level_0  index    EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                            
01/01/2013 GoalsScored       16     16  117.0      121    104.0        51.0
('01/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
01/01/2013 MatchesPlayed        5      5  42.0       40     40.0        18.0
('02/01/2013', 'GoalsScored')
                        level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                           
02/01/2013 GoalsScored       17     17  87.0      110    100.0       101.0
('02/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
02/01/2013 MatchesPlayed        6     

* row 선택하기

In [57]:
df2 = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [58]:
df2.head()

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [59]:
df2.loc[0] # 0은 인덱스를 의미, 여기서는 단순한 순서 ## column 인덱싱 

Month            08/01/2012
Stat          MatchesPlayed
EPL                      20
La Liga                  20
Serie A                  10
Bundesliga               10
Name: 0, dtype: object

In [60]:
df2 = df2.set_index('Month')
df2.head()

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [62]:
# index가 Month로 바뀌었으므로
df2.loc['08/01/2012']

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
08/01/2012,GoalsScored,57.0,60,21.0,23.0


In [63]:
# 0번째
df2.iloc[0]

Stat          MatchesPlayed
EPL                      20
La Liga                  20
Serie A                  10
Bundesliga               10
Name: 08/01/2012, dtype: object

In [64]:
df2 = df2.reset_index()

* multiple index

In [65]:
df2 = df2.set_index(['Month', 'Stat'])
df2.head()

,,EPL,La Liga,Serie A,Bundesliga
Month,Stat,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [66]:
df2.iloc[0]

EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: (08/01/2012, MatchesPlayed), dtype: float64

In [67]:
# multi index의 경우 ','로 구별하여 인덱싱 가능
df2.loc['08/01/2012', 'MatchesPlayed']

EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: (08/01/2012, MatchesPlayed), dtype: float64

* grouping with multi index

In [68]:
df2.head()

,,EPL,La Liga,Serie A,Bundesliga
Month,Stat,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [215]:
stat_group = df2.groupby(level = 'Stat')
for name, group in stat_group:
    print(name)
    print()
    print(group)

GoalsScored

                          EPL  La Liga  Serie A  Bundesliga
Month      Stat                                            
08/01/2012 GoalsScored  57.00       60    21.00       23.00
09/01/2012 GoalsScored 111.00      112   133.00      135.00
10/01/2012 GoalsScored  95.00       88    97.00       77.00
11/01/2012 GoalsScored 121.00      116   120.00      137.00
12/01/2012 GoalsScored 183.00      109   125.00       72.00
01/01/2013 GoalsScored 117.00      121   104.00       51.00
02/01/2013 GoalsScored  87.00      110   100.00      101.00
03/01/2013 GoalsScored  91.00      101    99.00      106.00
04/01/2013 GoalsScored 105.00      127   102.00      104.00
05/01/2013 GoalsScored  96.00      109   102.00       92.00
06/01/2013 GoalsScored    nan       80      nan         nan
MatchesPlayed

                           EPL  La Liga  Serie A  Bundesliga
Month      Stat                                             
08/01/2012 MatchesPlayed 20.00       20    10.00       10.00
09/01/201

In [71]:
# group의 sum method를 이용
# 각 리그당 경기 수와 골을 나타냄
stat_group.sum()

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,1063.0,1133,1003.0,898.0
MatchesPlayed,380.0,380,380.0,306.0


In [72]:
# 혹은 level을 이용하여 dataframe level에서 계산 가능
df2.sum(level = 'Stat')

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,1063.0,1133,1003.0,898.0
MatchesPlayed,380.0,380,380.0,306.0


In [73]:
total_goal_df = stat_group.sum()
total_goal_df.head()

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,1063.0,1133,1003.0,898.0
MatchesPlayed,380.0,380,380.0,306.0


###  각 리그의 경기당 평균골 구하기

* 평균 골 Series 생성

In [74]:
goals_per_game_df = total_goal_df.loc['GoalsScored'] / total_goal_df.loc['MatchesPlayed']
goals_per_game_df

EPL           2.797368
La Liga       2.981579
Serie A       2.639474
Bundesliga    2.934641
dtype: float64

* dataframe으로 변경

In [75]:
pd.DataFrame(goals_per_game_df)

,0
EPL,2.797368
La Liga,2.981579
Serie A,2.639474
Bundesliga,2.934641


* Transpose하여 row, column 변경

In [76]:
goals_per_game_df = pd.DataFrame(goals_per_game_df).T
goals_per_game_df

,EPL,La Liga,Serie A,Bundesliga
0,2.797368,2.981579,2.639474,2.934641


* mapping dictionary로 인덱스 값 변경

In [77]:
goals_per_game_df = goals_per_game_df \
                        .rename(index = \
                                {0 : 'goals per game'})
goals_per_game_df

,EPL,La Liga,Serie A,Bundesliga
goals per game,2.797368,2.981579,2.639474,2.934641


* 새로운 row로 추가

In [78]:
total_goal_df.append(goals_per_game_df)

,EPL,La Liga,Serie A,Bundesliga
GoalsScored,1063.000000,1133.000000,1003.000000,898.000000
MatchesPlayed,380.000000,380.000000,380.000000,306.000000
goals per game,2.797368,2.981579,2.639474,2.934641


* 소수점 2자리까지 출력하도록 설정

In [79]:
pd.options.display.float_format='{:.2f}'.format
total_goal_df.append(goals_per_game_df)

,EPL,La Liga,Serie A,Bundesliga
GoalsScored,1063.00,1133.00,1003.00,898.00
MatchesPlayed,380.00,380.00,380.00,306.00
goals per game,2.80,2.98,2.64,2.93


### Formating
  - https://pyformat.info/ 참고

### aggregate method
 - groupby 객체에 aggregate(집합) 함수 적용하여 그룹별 데이터 확인 가능

In [80]:
print(stat_group)
for name, group in stat_group:
    print(name)
    print(group)

GoalsScored
                          EPL  La Liga  Serie A  Bundesliga
Month      Stat                                            
08/01/2012 GoalsScored  57.00       60    21.00       23.00
09/01/2012 GoalsScored 111.00      112   133.00      135.00
10/01/2012 GoalsScored  95.00       88    97.00       77.00
11/01/2012 GoalsScored 121.00      116   120.00      137.00
12/01/2012 GoalsScored 183.00      109   125.00       72.00
01/01/2013 GoalsScored 117.00      121   104.00       51.00
02/01/2013 GoalsScored  87.00      110   100.00      101.00
03/01/2013 GoalsScored  91.00      101    99.00      106.00
04/01/2013 GoalsScored 105.00      127   102.00      104.00
05/01/2013 GoalsScored  96.00      109   102.00       92.00
06/01/2013 GoalsScored    nan       80      nan         nan
MatchesPlayed
                           EPL  La Liga  Serie A  Bundesliga
Month      Stat                                             
08/01/2012 MatchesPlayed 20.00       20    10.00       10.00
09/01/2012 

In [81]:
stat_group.count()

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,10,11,10,10
MatchesPlayed,10,11,10,10


In [82]:
stat_group.aggregate(len) ## len 함수로 모으기 ## 함수를 파라미터로 적용가능 

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,11.00,11,11.00,11.00
MatchesPlayed,11.00,11,11.00,11.00


In [83]:
stat_group.aggregate(np.sum) 

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,1063.00,1133,1003.00,898.00
MatchesPlayed,380.00,380,380.00,306.00


* multiple function 적용
  - 복수개의 column 레벨로 데이터 생성

In [84]:
stat_group.aggregate([np.sum, np.mean, np.size]) ## 함수를 리스트로 줄 수 있음

EPL              La Liga             Serie A               \
                  sum   mean  size     sum   mean size     sum   mean  size   
Stat                                                                          
GoalsScored   1063.00 106.30 11.00    1133 103.00   11 1003.00 100.30 11.00   
MatchesPlayed  380.00  38.00 11.00     380  34.55   11  380.00  38.00 11.00   

              Bundesliga              
                     sum  mean  size  
Stat                                  
GoalsScored       898.00 89.80 11.00  
MatchesPlayed     306.00 30.60 11.00

### transform 함수

* 가상 sales data 실습
 - excel로 구성된 가상의 판매 데이터 이용하여 transformation 실습
 - excel 데이터를 읽기 위해서 xlrd 모듈이 필요
   - pip install xlrd

In [85]:
import pandas as pd
df = pd.read_excel("data/sales_transactions.xlsx")

In [86]:
df.head(13)

,account,name,order,sku,quantity,unit price,ext price
0,383080,Will LLC,10001,B1-20000,7,33.69,235.83
1,383080,Will LLC,10001,S1-27722,11,21.12,232.32
2,383080,Will LLC,10001,B1-86481,3,35.99,107.97
3,412290,Jerde-Hilpert,10005,S1-06532,48,55.82,2679.36
4,412290,Jerde-Hilpert,10005,S1-82801,21,13.62,286.02
5,412290,Jerde-Hilpert,10005,S1-06532,9,92.55,832.95
6,412290,Jerde-Hilpert,10005,S1-47412,44,78.91,3472.04
7,412290,Jerde-Hilpert,10005,S1-27722,36,25.42,915.12
8,218895,Kulas Inc,10006,S1-27722,32,95.66,3061.12
9,218895,Kulas Inc,10006,B1-33087,23,22.55,518.65


In [87]:
df.groupby('order').sum()

,account,quantity,unit price,ext price
order,,,,
10001,1149240,21,90.80,576.12
10005,2061450,158,266.32,8185.49
10006,875580,57,262.69,3724.49


In [88]:
# ext price 컬럼만 sum
df.groupby('order')["ext price"].sum() ## 시리즈 

order
10001    576.12
10005   8185.49
10006   3724.49
Name: ext price, dtype: float64

* 위의 정보를 다시 원래의 frame에 합치려면?
 - 아래와 같이 다시 merge 하여 합칠 수도 있음 #나중에 
 - 가능은 하지만, 더 쉽게 transform으로 가능

In [89]:
order_total = df.groupby('order')["ext price"].sum().rename("total").reset_index()
order_total

,order,total
0,10001,576.12
1,10005,8185.49
2,10006,3724.49


In [90]:
result = df.merge(order_total, on='order') ## 오더에 대한 값 합한게 order_total , 조건명시를 on으로 함 
result["portion"] = result["ext price"] / result["total"] ## merge는 그룹으로 나눴을ㄸㅐ 개수가 줄어드는데 다시합칠때 갯수만큼 반복해줌 

In [91]:
result

,account,name,order,sku,quantity,unit price,ext price,total,portion
0,383080,Will LLC,10001,B1-20000,7,33.69,235.83,576.12,0.41
1,383080,Will LLC,10001,S1-27722,11,21.12,232.32,576.12,0.40
2,383080,Will LLC,10001,B1-86481,3,35.99,107.97,576.12,0.19
3,412290,Jerde-Hilpert,10005,S1-06532,48,55.82,2679.36,8185.49,0.33
4,412290,Jerde-Hilpert,10005,S1-82801,21,13.62,286.02,8185.49,0.03
5,412290,Jerde-Hilpert,10005,S1-06532,9,92.55,832.95,8185.49,0.10
6,412290,Jerde-Hilpert,10005,S1-47412,44,78.91,3472.04,8185.49,0.42
7,412290,Jerde-Hilpert,10005,S1-27722,36,25.42,915.12,8185.49,0.11
8,218895,Kulas Inc,10006,S1-27722,32,95.66,3061.12,3724.49,0.82
9,218895,Kulas Inc,10006,B1-33087,23,22.55,518.65,3724.49,0.14


* **transform 함수 사용!**
 - group by 후 transform을 사용하면 원본의 row를 유지한 상태에서 transform에 전달된 함수를 호출하여 aggregation
 - row의 개수가 유지되므로 바로 cloumn 추가 방법을 사용하여 원본 dataframe에 추가 가능

In [93]:
df.groupby('order')["ext price"].transform(np.sum)

0     576.12
1     576.12
2     576.12
3    8185.49
4    8185.49
5    8185.49
6    8185.49
7    8185.49
8    3724.49
9    3724.49
10   3724.49
11   3724.49
Name: ext price, dtype: float64

In [94]:
df['total'] = df.groupby('order')["ext price"].transform(np.sum)
df["portion"] = df["ext price"] / df["total"]
df

,account,name,order,sku,quantity,unit price,ext price,total,portion
0,383080,Will LLC,10001,B1-20000,7,33.69,235.83,576.12,0.41
1,383080,Will LLC,10001,S1-27722,11,21.12,232.32,576.12,0.40
2,383080,Will LLC,10001,B1-86481,3,35.99,107.97,576.12,0.19
3,412290,Jerde-Hilpert,10005,S1-06532,48,55.82,2679.36,8185.49,0.33
4,412290,Jerde-Hilpert,10005,S1-82801,21,13.62,286.02,8185.49,0.03
5,412290,Jerde-Hilpert,10005,S1-06532,9,92.55,832.95,8185.49,0.10
6,412290,Jerde-Hilpert,10005,S1-47412,44,78.91,3472.04,8185.49,0.42
7,412290,Jerde-Hilpert,10005,S1-27722,36,25.42,915.12,8185.49,0.11
8,218895,Kulas Inc,10006,S1-27722,32,95.66,3061.12,3724.49,0.82
9,218895,Kulas Inc,10006,B1-33087,23,22.55,518.65,3724.49,0.14


###  pivot 
 - dataframe의 형태를 변경 분석하기 쉬운 형태로 바꿀수 있음 
 - 변경 시, index, column, data를 명시

* ** 식물실험 데이터**
 - ctrl, tr1, tr2 환경에서의 회차별 weight 관측 데이터

In [96]:
plant_df = pd.read_csv('data/PlantGrowth.csv')
plant_df.head(30)

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [97]:
plant_df
# control(ctrl)하여 실험한 값과, 두 개의 다른 조건(tr1, tr2)을 주었을 때의 실험에 대한 결과 
# data가 stacked 됨. (말 그대로 쌓여있음)
# 데이터 분석하기 편리하지 않음

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [98]:
# 아래와 같이 filtering하여 각각의 결과를 뽑을 수 있으나, pivot, unstack을 이용하여 분석하기 용이하도록 변형 가능
plant_df[plant_df['group'] == 'ctrl']

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [99]:
# dataframe reshaping
# 각각 index, columns, values로 명시하여 data frame의 shape을 변경 가능
plant_df.pivot(index = 'observation', 
               columns = 'group', 
               values = 'weight')

group,ctrl,trt1,trt2
observation,,,
1,4.17,4.81,6.31
2,5.58,4.17,5.12
3,5.18,4.41,5.54
4,6.11,3.59,5.50
5,4.50,5.87,5.37
6,4.61,3.83,5.29
7,5.17,6.03,4.92
8,4.53,4.89,6.15
9,5.33,4.32,5.80


In [100]:
# pivot_table 으로도 동일 결과 가능
pd.pivot_table(plant_df, values = 'weight', index = 'observation', columns='group')

group,ctrl,trt1,trt2
observation,,,
1,4.17,4.81,6.31
2,5.58,4.17,5.12
3,5.18,4.41,5.54
4,6.11,3.59,5.50
5,4.50,5.87,5.37
6,4.61,3.83,5.29
7,5.17,6.03,4.92
8,4.53,4.89,6.15
9,5.33,4.32,5.80


In [101]:
pd.pivot_table(plant_df, values = 'weight', columns='group') ## 평균을 내버림 

group,ctrl,trt1,trt2
weight,5.03,4.66,5.53


In [102]:
pd.pivot_table(plant_df, values = 'weight', columns='observation') ## 데이터값을 어떤것을 가져올지 모를때 평균을 내여 가져옴 

observation,1,2,3,4,5,6,7,8,9,10
weight,5.10,4.96,5.04,5.07,5.25,4.58,5.37,5.19,5.15,5.03


* **pivot, pivot_table의 차이**
 - pivot_table 
   - pivot 결과 생성 시, 값 선택이 vague한 경우, aggfunc 사용하여 채움 =기본값이 mean


In [103]:
from collections import OrderedDict

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))
df = pd.DataFrame(table)

In [104]:
df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item1,Gold,3,3
3,Item1,Silver,4,4


In [105]:
df.pivot(index='Item', columns='CType', values='USD')

CType,Bronze,Gold,Silver
Item,,,
Item0,2.00,1.00,nan
Item1,nan,3.00,4.00


In [106]:
df.pivot(index='Item', columns='CType')

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0   2.00 1.00    nan   2.00 1.00    nan
Item1    nan 3.00   4.00    nan 3.00   4.00

In [107]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))

df = pd.DataFrame(table)

df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item0,Gold,3,3
3,Item1,Silver,4,4


In [256]:
# Item0과 Gold에 대해서 1, 3중 어떤 것을 채워야 할지 알 수 없음
df.pivot(index='Item', columns='CType', values='USD')

KeyError: 'Item'

In [109]:
df.pivot_table(index='Item', 
               columns='CType', 
               values='USD',
              aggfunc = np.sum)

CType,Bronze,Gold,Silver
Item,,,
Item0,2.00,4.00,nan
Item1,nan,nan,4.00


###  stack & unstack
 - stack : inner-most column -> inner-most index
 - unstack : inner-most index -> inner-most column
 
 - 둘은 역의 관계에 있음

In [ ]:
plant_df

In [ ]:
stacked = plant_df.set_index(['group', 'observation'])
stacked

In [ ]:
# 가장 바깥 레벨의 row -> column으로 이동
stacked.unstack()

In [ ]:
# 레벨 0의 row -> column으로 이동
stacked.unstack(0)

# 레벨의 순서가 아닌, 네임으로도 명시 가능
# stacked.unstack(level = 'group')

In [ ]:
# 컬럼을 다시 가장 하위의 row로 이동
stacked.unstack(0).stack()

### Docstring 

In [218]:
def add(a, b):
    '''calculate sum of two numeric values 
    
    parameters:
        a, b: any numeric value
    
    returns:
        sumation of a and b
        
    exceptions:
        '''
    if a ==0:
        raise Exception('invalid')
        
    return a+b

### Concat ## 두개의 데이터프래임을 붙이는것

In [216]:
import pandas as pd
import numpy as np

In [219]:
df1 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [220]:
df1

,key1,value1
0,0,-1.20
1,1,0.96
2,2,0.25
3,3,-0.06
4,4,0.48
5,5,-0.11
6,6,-0.63
7,7,1.25
8,8,-0.33
9,9,0.79


In [221]:
df2 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [222]:
df2

,key1,value1
0,0,0.36
1,1,-1.56
2,2,-0.82
3,3,-2.07
4,4,-1.50
5,5,-0.21
6,6,-0.46
7,7,-0.73
8,8,-1.24
9,9,-0.84


In [223]:
# axis=0 기본적으로 row를 따라 합침
pd.concat([df1, df2]) ## 인덱스가 중복되어서 데이터 모두 쌓임 

,key1,value1
0,0,-1.20
1,1,0.96
2,2,0.25
3,3,-0.06
4,4,0.48
5,5,-0.11
6,6,-0.63
7,7,1.25
8,8,-0.33
9,9,0.79


In [224]:
# axis=1로 주어 column을 따라 합치는 것도 가능
pd.concat([df1, df2], axis=1)

,key1,value1,key1,value1
0,0,-1.20,0,0.36
1,1,0.96,1,-1.56
2,2,0.25,2,-0.82
3,3,-0.06,3,-2.07
4,4,0.48,4,-1.50
5,5,-0.11,5,-0.21
6,6,-0.63,6,-0.46
7,7,1.25,7,-0.73
8,8,-0.33,8,-1.24
9,9,0.79,9,-0.84


In [225]:
# axis=0 기본적으로 row를 따라 합침
# 이때 기존의 인덱스를 무시하려면 ignore_index 파라미터를 True로 변경
pd.concat([df1, df2], ignore_index=True) ## 인덱스는 번호를 따라가지만 키에 저장된 정보는 그대로 있음 

,key1,value1
0,0,-1.20
1,1,0.96
2,2,0.25
3,3,-0.06
4,4,0.48
5,5,-0.11
6,6,-0.63
7,7,1.25
8,8,-0.33
9,9,0.79


* column 네임이 다를 경우

In [228]:
df3 = pd.DataFrame({'key2' : np.arange(10), 'value2' : np.random.randn(10)})

In [227]:
# 없는 컬럼에 대해서는 nan으로 채움
pd.concat([df1, df3], ignore_index=True)

,key1,key2,value1,value2
0,0.00,nan,-1.20,nan
1,1.00,nan,0.96,nan
2,2.00,nan,0.25,nan
3,3.00,nan,-0.06,nan
4,4.00,nan,0.48,nan
5,5.00,nan,-0.11,nan
6,6.00,nan,-0.63,nan
7,7.00,nan,1.25,nan
8,8.00,nan,-0.33,nan
9,9.00,nan,0.79,nan


* index를 설정하고 concat

In [229]:
df11 = df1.set_index('key1')

In [230]:
df22 = df2.set_index('key1')

In [231]:
pd.concat([df11, df22])

,value1
key1,
0,-1.20
1,0.96
2,0.25
3,-0.06
4,0.48
5,-0.11
6,-0.63
7,1.25
8,-0.33


In [232]:
result = pd.concat([df11, df22], axis=1)
result

,value1,value1
key1,,
0,-1.20,0.36
1,0.96,-1.56
2,0.25,-0.82
3,-0.06,-2.07
4,0.48,-1.50
5,-0.11,-0.21
6,-0.63,-0.46
7,1.25,-0.73
8,-0.33,-1.24


### Merge & Join

In [233]:
df1 = pd.DataFrame({'key1' : np.arange(8), 
                    'key2' : ['LA', 'LA', 'SF', 'SF', 'SF', 'NY', 'NY', 'LV'], 
                    'value1' : np.random.randn(8)})

In [234]:
df1

,key1,key2,value1
0,0,LA,0.92
1,1,LA,1.78
2,2,SF,-0.74
3,3,SF,-1.48
4,4,SF,-1.22
5,5,NY,1.14
6,6,NY,0.60
7,7,LV,-1.62


In [235]:
df2 = pd.DataFrame({'key1' : [1, 1, 2, 2, 2, 3, 3], 
                    'key2' : ['LA', 'LA', 'KC', 'KC', 'NY', 'SF', 'SF'], 
                    'value1' : np.random.randn(7)})

In [236]:
df2

,key1,key2,value1
0,1,LA,-0.74
1,1,LA,0.83
2,2,KC,0.08
3,2,KC,1.55
4,2,NY,0.87
5,3,SF,-0.09
6,3,SF,-1.05


* on
 - join 하고자 하는 대상이 되는 column 명시

In [237]:
pd.merge(df1, df2, on='key1') ## 두개의 프래임을 merge할때 기준을 on 으로 명시 

,key1,key2_x,value1_x,key2_y,value1_y
0,1,LA,1.78,LA,-0.74
1,1,LA,1.78,LA,0.83
2,2,SF,-0.74,KC,0.08
3,2,SF,-0.74,KC,1.55
4,2,SF,-0.74,NY,0.87
5,3,SF,-1.48,SF,-0.09
6,3,SF,-1.48,SF,-1.05


In [238]:
pd.merge(df1, df2, on='key2') ## inner로 merge가 되어있기 땜에 key2 기준이 되어 SF 에 대한 값들을 모두 명시하기에 row가 늘어남 

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,0.92,1,-0.74
1,0,LA,0.92,1,0.83
2,1,LA,1.78,1,-0.74
3,1,LA,1.78,1,0.83
4,2,SF,-0.74,3,-0.09
5,2,SF,-0.74,3,-1.05
6,3,SF,-1.48,3,-0.09
7,3,SF,-1.48,3,-1.05
8,4,SF,-1.22,3,-0.09
9,4,SF,-1.22,3,-1.05


* how parameter
 - SQL의 join과 같이 동작
 - inner : 기본 값, 일치 하는 값들로만 조인 ## 교집합만 새로운 테이블에 조인
 - left : left outer join과 같이 동작 ## 왼쪽에 있는 df1에 있는 정보를 모두 가져오고 df2에 매치되는 값(교집합값) 만 가져옴, 나머지는 Nan으로 반환 
 - right : right outer join과 같이 동작 ## 오른쪽에 있는 df2 의 정보를 모두 가져오고 기준으로 함,df1은 매치되는 값만 가져옴 
 - outer : full outer join과 같이 동작, 즉 left + right ## 합집합으로 모든 정보를 다가져옴, 없는 값은 Nan으로 반환 

* left join
 - 왼쪽 df 기준으로 join, 없는 경우 nan 할당

In [239]:
pd.merge(df1, df2, on='key2', how='left')

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,0.92,1.00,-0.74
1,0,LA,0.92,1.00,0.83
2,1,LA,1.78,1.00,-0.74
3,1,LA,1.78,1.00,0.83
4,2,SF,-0.74,3.00,-0.09
5,2,SF,-0.74,3.00,-1.05
6,3,SF,-1.48,3.00,-0.09
7,3,SF,-1.48,3.00,-1.05
8,4,SF,-1.22,3.00,-0.09
9,4,SF,-1.22,3.00,-1.05


* right join
 - 오른쪽 df 기준으로 join, 없는 경우 nan 할당

In [240]:
pd.merge(df1, df2, on='key2', how='right')

,key1_x,key2,value1_x,key1_y,value1_y
0,0.00,LA,0.92,1,-0.74
1,1.00,LA,1.78,1,-0.74
2,0.00,LA,0.92,1,0.83
3,1.00,LA,1.78,1,0.83
4,2.00,SF,-0.74,3,-0.09
5,3.00,SF,-1.48,3,-0.09
6,4.00,SF,-1.22,3,-0.09
7,2.00,SF,-0.74,3,-1.05
8,3.00,SF,-1.48,3,-1.05
9,4.00,SF,-1.22,3,-1.05


* full join
 - left join 과 right join union

In [241]:
pd.merge(df1, df2, on='key2', how='outer') ## outer = left + right 

,key1_x,key2,value1_x,key1_y,value1_y
0,0.00,LA,0.92,1.00,-0.74
1,0.00,LA,0.92,1.00,0.83
2,1.00,LA,1.78,1.00,-0.74
3,1.00,LA,1.78,1.00,0.83
4,2.00,SF,-0.74,3.00,-0.09
5,2.00,SF,-0.74,3.00,-1.05
6,3.00,SF,-1.48,3.00,-0.09
7,3.00,SF,-1.48,3.00,-1.05
8,4.00,SF,-1.22,3.00,-0.09
9,4.00,SF,-1.22,3.00,-1.05


* **multi key 조인**
 - join 하고자 할 column 리스트로 구성

In [242]:
pd.merge(df1, df2, on=['key1', 'key2'])

,key1,key2,value1_x,value1_y
0,1,LA,1.78,-0.74
1,1,LA,1.78,0.83
2,3,SF,-1.48,-0.09
3,3,SF,-1.48,-1.05


In [243]:
pd.merge(df1, df2, on=['key1', 'key2'], how='left')

,key1,key2,value1_x,value1_y
0,0,LA,0.92,nan
1,1,LA,1.78,-0.74
2,1,LA,1.78,0.83
3,2,SF,-0.74,nan
4,3,SF,-1.48,-0.09
5,3,SF,-1.48,-1.05
6,4,SF,-1.22,nan
7,5,NY,1.14,nan
8,6,NY,0.60,nan
9,7,LV,-1.62,nan


In [244]:
# 나머지 column의 이름이 겹칠 경우 suffix 지정 가능
pd.merge(df1, df2, on=['key1', 'key2'], how='left', 
         suffixes=('_left', '_right'))

,key1,key2,value1_left,value1_right
0,0,LA,0.92,nan
1,1,LA,1.78,-0.74
2,1,LA,1.78,0.83
3,2,SF,-0.74,nan
4,3,SF,-1.48,-0.09
5,3,SF,-1.48,-1.05
6,4,SF,-1.22,nan
7,5,NY,1.14,nan
8,6,NY,0.60,nan
9,7,LV,-1.62,nan


* index가 있는 경우 merge
 - left_index : 왼쪽 프레임에서 인덱스를 사용할 경우 True 명시
 - right_index : 오른쪽 프레임에서 인덱스를 사용할 경우 True 명시

In [245]:
df11 = df1.set_index('key1')
df11

,key2,value1
key1,,
0,LA,0.92
1,LA,1.78
2,SF,-0.74
3,SF,-1.48
4,SF,-1.22
5,NY,1.14
6,NY,0.60
7,LV,-1.62


In [246]:
df22 = df2.set_index('key1')
df22

,key2,value1
key1,,
1,LA,-0.74
1,LA,0.83
2,KC,0.08
2,KC,1.55
2,NY,0.87
3,SF,-0.09
3,SF,-1.05


In [247]:
# index끼리 조인할 것이라고 명시 가능
pd.merge(df11, df22, left_index=True, right_index=True)

,key2_x,value1_x,key2_y,value1_y
key1,,,,
1,LA,1.78,LA,-0.74
1,LA,1.78,LA,0.83
2,SF,-0.74,KC,0.08
2,SF,-0.74,KC,1.55
2,SF,-0.74,NY,0.87
3,SF,-1.48,SF,-0.09
3,SF,-1.48,SF,-1.05
